In [308]:
import json
import requests
from config import api_key, lw_key
import pandas as pd
from datetime import date, timedelta, datetime
import geopy
import pgeocode
nomi = pgeocode.Nominatim('US')


In [32]:
url= "https://api.worldweatheronline.com/premium/v1/weather.ashx?"
past_url = "https://api.worldweatheronline.com/premium/v1/past-weather.ashx?"
# query_url =f"{url}appid={api_key}&zip="

In [17]:
zipcode = input("Enter 5-digit Zipcode:")

Enter 5-digit Zipcode:21230


In [ ]:
# zip_df = nomi.query_postal_code(zipcode)


In [ ]:
# pandas apply pgeocode to ZIPCODE1
# md_zips = df['ZIPCODE1'].apply(nomi.query_postal_code)
# md_zips.head()

In [277]:
response = requests.get(f"{url}key={lw_key}&q={zipcode}&num_of_days=8&tp=24&cc=no&mca=no&aqi=yes&format=json").json()
response = response['data']


In [292]:
weather_dict ={
    "Dates":[],
    "Cloudcover":[],
    "Humidity":[],
    "PrecipInch":[],
    "Pressure":[],
    "FeelsLike":[],
    "HeatIndex":[],
    "MaxTemp":[],
    "MinTemp":[],
    "SunHours":[],
    "UVIndex":[],
}

weather_dict["Dates"]= ([response["weather"][i]["date"] for i in range(len(response["weather"]))])
weather_dict["Cloudcover"]=([response["weather"][i]["hourly"][0]["cloudcover"] for i in range(len(response["weather"]))])
weather_dict["Humidity"]= ([response["weather"][i]["hourly"][0]["humidity"] for i in range(len(response["weather"]))])
weather_dict["PrecipInch"]= ([response["weather"][i]["hourly"][0]["precipInches"] for i in range(len(response["weather"]))])
weather_dict["Pressure"]= ([response["weather"][i]["hourly"][0]["pressure"] for i in range(len(response["weather"]))])
weather_dict["FeelsLike"]= ([response["weather"][i]["hourly"][0]["FeelsLikeF"] for i in range(len(response["weather"]))])
weather_dict["HeatIndex"]= ([response["weather"][i]["hourly"][0]["HeatIndexF"] for i in range(len(response["weather"]))])
weather_dict["MaxTemp"]= ([response["weather"][i]["maxtempF"] for i in range(len(response["weather"]))])
weather_dict["MinTemp"]=([response["weather"][i]["mintempF"] for i in range(len(response["weather"]))])
weather_dict["SunHours"]=([response["weather"][i]["sunHour"] for i in range(len(response["weather"]))])
weather_dict["UVIndex"]= ([response["weather"][i]["hourly"][0]["uvIndex"] for i in range(len(response["weather"]))])
weather_df= pd.DataFrame.from_dict(weather_dict,orient='index').transpose()
weather_df= weather_df.apply(pd.to_numeric, errors= 'ignore')
weather_df["TempDelta"] = weather_df.MaxTemp - weather_df.MinTemp
weather_df["BarChange"] =weather_df["Pressure"].pct_change()
weather_df["HeatChange"] =weather_df["HeatIndex"].pct_change()
weather_df["HumChange"] =weather_df["Humidity"].pct_change()
weather_df= weather_df.iloc[1:]
weather_df.head(10)


,Dates,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex,TempDelta,BarChange,HeatChange,HumChange
1,2021-04-15,55,70,0.2,1008,53,55,62,48,9.1,3,14,-0.007874,-0.017857,-0.176471
2,2021-04-16,43,61,0.0,1008,42,48,54,44,7.8,4,10,0.000000,-0.127273,-0.128571
3,2021-04-17,61,65,0.0,1011,46,49,53,44,6.6,4,9,0.002976,0.020833,0.065574
4,2021-04-18,47,62,0.0,1012,50,52,58,45,7.9,1,13,0.000989,0.061224,-0.046154
5,2021-04-19,60,64,0.0,1014,53,54,61,47,6.6,3,14,0.001976,0.038462,0.032258
6,2021-04-20,35,60,0.0,1016,56,57,67,48,10.4,4,19,0.001972,0.055556,-0.062500
7,2021-04-21,37,57,0.0,1014,51,54,58,48,10.4,3,10,-0.001969,-0.052632,-0.050000


In [324]:
dates= input("Dates you had a migraine (YYYY-MM-DD):")


Dates you had a migraine (YYYY-MM-DD):2021-04-10, 2021-04-11


In [329]:
dates=dates.split(",")
dates

['2021-04-10', ' 2021-04-11']

In [333]:
datess=[(datetime.strptime(i) - timedelta(1)) for i in dates]


TypeError: strptime() takes exactly 2 arguments (1 given)

In [46]:
for date in dates:
    responses = requests.get(f"{past_url}key={lw_key}&q={zipcode}&date={date}&tp=24&mca=no&aqi=yes&format=json").json()
    print(json.dumps(responses, indent=4, sort_keys=True))

{
    "data": {
        "request": [
            {
                "query": "21230",
                "type": "Zipcode"
            }
        ],
        "weather": [
            {
                "astronomy": [
                    {
                        "moon_illumination": "59",
                        "moon_phase": "First Quarter",
                        "moonrise": "12:39 PM",
                        "moonset": "03:20 AM",
                        "sunrise": "07:06 AM",
                        "sunset": "07:21 PM"
                    }
                ],
                "avgtempC": "12",
                "avgtempF": "54",
                "date": "2021-03-22",
                "hourly": [
                    {
                        "DewPointC": "1",
                        "DewPointF": "34",
                        "FeelsLikeC": "11",
                        "FeelsLikeF": "52",
                        "HeatIndexC": "11",
                        "HeatIndexF": "52",
                 

In [ ]:
# weather_df[[f"{date.today()}", f"{date.today()+timedelta(1)}", f"{date.today()+timedelta(2)}", f"{date.today()+timedelta(3)}", f"{date.today()+timedelta(4)}", f"{date.today()+timedelta(5)}", f"{date.today()+timedelta(6)}", f"{date.today()+timedelta(7)}"]]= pd.DataFrame(weather_df.Pressure.to_list(), index = weather_df.index)
# forecast = weather_df.drop(columns= ["Dates","Pressure"])
# columns = ["Date 1", "Date 2", "Date 3", "Date 4", "Date 5", "Date 6", "Date 7", "Date 8"], index = weather_df.index
# rural_cats = rural_cat[["Category 1", "Category 2", "Category 3"]]
# rural_clean = rural.merge(rural_cats, left_index = True,right_index = True)
# forecast.head()